In [ ]:
# Import libraries
import os
# Install selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import urllib.request
from bs4 import BeautifulSoup
import moviepy.editor as mp 
import speech_recognition as sr
import time

import pandas as pd

### Download chromedriver and save the .exe file into your folder

chromedriver.exe (Download from: https://chromedriver.chromium.org/)

In [ ]:
# Read the excel file containing video_url
# Change the file name to the file you want to use
df = pd.read_excel("data/videos_all.xlsx")

In [ ]:
# Enter the local path of the chromedriver file
driver = webdriver.Chrome(r'/Users/Flame/Dropbox/Summer/QMB6930/chromedriver')
# A new chrome window will pop out

# Set directory
os.chdir('/Users/Flame/Dropbox/Summer/QMB6930') # Enter your folder path

# Create a temp directory
if not os.path.exists("./temp"):
    os.makedirs("./temp")

In [ ]:
# This block of code returns transcript of videos
# This may run for hours depending on the amount of videos

# Create an empty list for the results
result_l = []

i = 0
for url in df['video_url']:
    
    i += 1
    driver.get(url)
    
    # give chrome some time to load the page
    time.sleep(2)
    
    # 1. Get page_source
    # 2. Parse HTML
    # 3. Find the node for the video
    video_url = BeautifulSoup(driver.page_source, 'html.parser').find("video")
    
    try:
        # Download video and save as "./temp/temp.mp4"
        urllib.request.urlretrieve(video_url["src"], './temp/temp.mp4')
        
        # Rip audio from the video
        clip = mp.VideoFileClip('./temp/temp.mp4')
        clip.audio.write_audiofile('./temp/temp.wav')
        
        # Run Speech Recognition
        r = sr.Recognizer()
        audio_file = sr.AudioFile('./temp/temp.wav')
        
        with audio_file as source:
            r.adjust_for_ambient_noise(source)
            audio = r.record(source)
            
        result = r.recognize_google(audio)

    except:
        result = ''
    
    # Save results to the list
    result_l.append(result)
    # This will show up every time a transcription is done
    print('video {} is done.'.format(str(i)))

In [ ]:
# Close the webdriver
driver.quit()

In [ ]:
# Check the length 
# Make sure the outcome is True, meaning all transcription is done
len(result_l) == len(df['video_url'])

In [ ]:
# Add the list of transcripts to a new column
df['video_transcript'] = result_l

# Change video id and timestamp to str type
df = df.astype({"video_id": str, 'video_timestamp': str})

In [ ]:
# Save the new dataframe as an excel file (replace the original one)
df.to_excel("videos_all.xlsx",sheet_name='videos', index=False) 